## Setup

In [1]:
%%capture
!pip install -q accelerate>=0.27.2 peft>=0.9.0 bitsandbytes>=0.43.0 transformers>=4.38.2 trl>=0.7.11 sentencepiece>=0.1.99
!pip install -q sentence-transformers>=3.0.0 mteb>=1.1.2 datasets>=2.18.0

In [1]:
from datasets import load_dataset, Dataset
from sentence_transformers import SentenceTransformer, models
from sentence_transformers import losses
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.training_args import SentenceTransformerTrainingArguments
from sentence_transformers.trainer import SentenceTransformerTrainer
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.datasets import DenoisingAutoEncoderDataset
from mteb import MTEB

import gc
import torch
import random
from tqdm import tqdm
import pandas as pd
import numpy as np

/Users/rahiakela/Documents/GitHub/genai-research-and-practice/hands-on-large-language-models/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Dataset

In [2]:
# Load MNLI dataset from GLUE
# 0 = entailment, 1 = neutral, 2 = contradiction
train_dataset = load_dataset("glue", "mnli", split="train").select(range(50_000))
train_dataset = train_dataset.remove_columns("idx")

In [3]:
train_dataset[2]

{'premise': 'One of our number will carry out your instructions minutely.',
 'hypothesis': 'A member of my team will execute your orders with immense precision.',
 'label': 0}

## Model

In [4]:
device = torch.device("mps")
# Use a base model
embedding_model = SentenceTransformer("bert-base-uncased", device=device)
# embedding_model=embedding_model.to(device)

No sentence-transformers model found with name bert-base-uncased. Creating a new one with mean pooling.


In [5]:
embedding_model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

## Loss Function

In [6]:
# Define the loss function. In soft-max loss, we will also need to explicitly set the number of labels.
train_loss = losses.SoftmaxLoss(
    model=embedding_model, 
    sentence_embedding_dimension=embedding_model.get_sentence_embedding_dimension(), 
    num_labels=3
)

## Evaluation

In [7]:
# Create an embedding similarity evaluator for stsb
val_sts = load_dataset("glue", "stsb", split="validation")
evaluator = EmbeddingSimilarityEvaluator(
    sentences1=val_sts["sentence1"],
    sentences2=val_sts["sentence2"],
    scores=[score/5 for score in val_sts["label"]],
    main_similarity="cosine",
)

## Training

In [8]:
# Define the training arguments
training_args = SentenceTransformerTrainingArguments(
    output_dir="base_embedding_model",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=100,
    fp16=False,
    eval_steps=100,
    logging_steps=100, 
)

In [9]:
# Train embedding model
trainer = SentenceTransformerTrainer(
    model=embedding_model, 
    args=training_args, 
    train_dataset=train_dataset, 
    loss=train_loss,
    evaluator=evaluator,
)

In [10]:
trainer.train()

  0%|          | 0/6250 [00:00<?, ?it/s]Column 'hypothesis' is at index 1, whereas a column with this name is usually expected at index 0. Note that the column order can be important for some losses, e.g. MultipleNegativesRankingLoss will always consider the first column as the anchor and the second as the positive, regardless of the dataset column names. Consider renaming the columns to match the expected order, e.g.:
dataset = dataset.select_columns(['hypothesis', 'entailment', 'contradiction'])
  2%|▏         | 100/6250 [00:34<30:32,  3.36it/s]

{'loss': 1.0957, 'grad_norm': 4.804290771484375, 'learning_rate': 5e-05, 'epoch': 0.02}


  3%|▎         | 200/6250 [01:03<28:43,  3.51it/s]

{'loss': 1.013, 'grad_norm': 3.2441935539245605, 'learning_rate': 4.9186991869918704e-05, 'epoch': 0.03}


  5%|▍         | 300/6250 [01:32<27:26,  3.61it/s]

...

{'loss': 0.8037, 'grad_norm': 11.284797668457031, 'learning_rate': 4.0243902439024395e-05, 'epoch': 0.21}


 22%|██▏       | 1400/6250 [07:40<27:17,  2.96it/s]

{'loss': 0.7958, 'grad_norm': 8.96224308013916, 'learning_rate': 3.943089430894309e-05, 'epoch': 0.22}


 23%|██▎       | 1433/6250 [07:50<24:00,  3.34it/s]

RuntimeError: MPS backend out of memory (MPS allocated: 6.74 GB, other allocations: 11.31 GB, max allowed: 18.13 GB). Tried to allocate 89.42 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).